In [35]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.ml.feature import StringIndexer

from pyspark.sql.functions import col, explode
import pandas as pd
import pyspark.sql.functions as func
from pyspark.mllib.evaluation import RegressionMetrics, RankingMetrics
from pyspark import SparkContext
# import findspark 
# findspark.init()  
# import pyspark

In [36]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.config("spark.driver.memory", "16G").getOrCreate()
#for later --conf spark.sql.catalogImplementation=hive 

In [37]:
trainSample = spark.read.parquet('train_sample1.parquet')
testSample = spark.read.parquet('test_sample1.parquet')
trainSample.createOrReplaceTempView('trainSample')
testSample.createOrReplaceTempView('testSample')

In [46]:
valSample = spark.read.parquet('val_sample3.parquet')

In [5]:
indexer_obj_1 = StringIndexer(inputCol="user_id", outputCol="user_id_numer").setHandleInvalid("keep")
indexer_model_1 = indexer_obj_1.fit(trainSample)
indexer_df_1 = indexer_model_1.transform(trainSample)

indexer_obj_2 = StringIndexer(inputCol="track_id", outputCol="track_id_numer").setHandleInvalid("keep")
indexer_model_2= indexer_obj_2.fit(indexer_df_1)
indexer_df_2 = indexer_model_2.transform(indexer_df_1)

train_df = indexer_df_2.drop('user_id')
train_df= train_df.drop('track_id')


In [47]:
val_df_1 = indexer_model_1.transform(valSample)
val_df_2= indexer_model_2.transform(val_df_1)

val_df = val_df_2.drop('user_id')
val_df= val_df.drop('track_id')

In [48]:
val_df

DataFrame[count: bigint, __index_level_0__: bigint, user_id_numer: double, track_id_numer: double]

In [9]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="user_id_numer", itemCol="track_id_numer", ratingCol="count",
          coldStartStrategy="drop")
model = als.fit(train_df)

#userRecs = model.recommendForAllUsers(10)

# Evaluate the model by computing the RMSE on the test data

# evaluator = RegressionEvaluator(metricName="rmse", labelCol="count",
#                                 predictionCol="prediction")
# rmse = evaluator.evaluate(predictions)

# print("Root-mean-square error = " + str(rmse))

# Generate top 10 song recommendations for each user

### ADJUST FOR 500 SONGS 

# Generate top 10 user recommendations for each song

#songRecs = model.recommendForAllItems(10)

In [ ]:
#### THIS WILL BE REMOVED ####

# Generate top 10 movie recommendations for a specified set of users
users = train_df.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = model.recommendForUserSubset(users, 10)

# Generate top 10 user recommendations for a specified set of movies
# songs = df_dropped.select(als.getItemCol()).distinct().limit(3)
# songSubSetRecs = model.recommendForItemSubset(songs, 10)

In [ ]:
## MIGHT NOT NEED ##

predictions = model.transform(test_df)
predictions.orderBy('prediction', ascending=False)

In [49]:
# for each user, sort track ids by count
val_true = val_df.orderBy('count')

# flatten to group by user id and get list of true track ids
val_true_flatten = val_true.groupby('user_id_numer').agg(func.collect_list('track_id_numer').alias("track_id_numer"))

# add to dictionary
val_true_dict = val_true_flatten.collect()
val_true_dict = [{r['user_id_numer']: r['track_id_numer']} for r in val_true_dict]
val_true_dict = dict((key,d[key]) for d in val_true_dict for key in d)
val_true_dict

Exception ignored in: <function JavaModelWrapper.__del__ at 0x7fd2f8e14160>
Traceback (most recent call last):
  File "/Users/harlanhutton/opt/anaconda3/lib/python3.8/site-packages/pyspark/mllib/common.py", line 137, in __del__
    self._sc._gateway.detach(self._java_model)
AttributeError: 'RankingMetrics' object has no attribute '_sc'
Exception ignored in: <function JavaModelWrapper.__del__ at 0x7fd2f8e14160>
Traceback (most recent call last):
  File "/Users/harlanhutton/opt/anaconda3/lib/python3.8/site-packages/pyspark/mllib/common.py", line 137, in __del__
    self._sc._gateway.detach(self._java_model)
AttributeError: 'RankingMetrics' object has no attribute '_sc'
Exception ignored in: <function JavaModelWrapper.__del__ at 0x7fd2f8e14160>
Traceback (most recent call last):
  File "/Users/harlanhutton/opt/anaconda3/lib/python3.8/site-packages/pyspark/mllib/common.py", line 137, in __del__
    self._sc._gateway.detach(self._java_model)
AttributeError: 'RankingMetrics' object has no at

{314352.0: [80289.0],
 320754.0: [1491.0],
 144345.0: [101379.0, 7876.0, 2576.0],
 209554.0: [24790.0],
 327135.0: [279.0],
 318899.0: [11051.0],
 38228.0: [106998.0, 84505.0],
 257834.0: [5251.0, 1748.0, 28356.0],
 305459.0: [21714.0],
 230409.0: [39283.0],
 191277.0: [6349.0],
 310372.0: [35043.0],
 172967.0: [1318.0],
 222536.0: [3329.0],
 183020.0: [5537.0],
 142896.0: [106998.0],
 302903.0: [208.0],
 303341.0: [106998.0, 17245.0, 67991.0],
 150508.0: [21858.0],
 274538.0: [19988.0],
 259239.0: [1417.0, 6919.0],
 279263.0: [5665.0],
 284521.0: [103.0],
 231190.0: [106998.0],
 310465.0: [39643.0],
 142084.0: [37.0],
 293951.0: [2662.0, 85544.0],
 106696.0: [501.0],
 248754.0: [2569.0],
 307804.0: [101.0],
 63137.0: [40057.0],
 122505.0: [7003.0],
 204949.0: [32510.0, 53124.0],
 230661.0: [20345.0],
 233218.0: [24.0],
 159194.0: [4265.0],
 220558.0: [106998.0],
 321044.0: [18028.0],
 289159.0: [508.0],
 119462.0: [38150.0],
 120784.0: [43496.0, 64175.0],
 331884.0: [43520.0],
 222109

In [ ]:
#https://stackoverflow.com/questions/59390481/how-to-implement-ranking-metrics-of-pyspark

In [53]:
val_preds = model.recommendForUserSubset(val_df,1)
val_preds_explode = val_preds.select(val_preds.user_id_numer,explode(val_preds.recommendations.track_id_numer))

val_preds_flatten = val_preds_explode.groupby('user_id_numer').agg(func.collect_list('col').alias("col"))

val_preds_dict = val_preds_flatten.collect()
val_preds_dict = [{r['user_id_numer']: r['col']} for r in val_preds_dict]
val_preds_dict = dict((key,d[key]) for d in val_preds_dict for key in d)

Evaluation Metric

In [61]:
labels_list = []

for user in val_preds_dict.keys():
    labels_list.append((val_preds_dict[user][0], [int(i) for i in val_true_dict[user]][0]))

labels = spark.sparkContext.parallelize(labels_list)
metrics = RankingMetrics(labels)
print(metrics.meanAveragePrecision)
    
# add predicted items and true items as tuples to list

# parallelize list
# call ranking metrics on list 

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 112.0 failed 1 times, most recent failure: Lost task 0.0 in stage 112.0 (TID 1626) (harlans-mbp executor driver): org.apache.spark.SparkException: 
Error from python worker:
  dyld: Library not loaded: /System/Library/Frameworks/CoreFoundation.framework/Versions/A/CoreFoundation
    Referenced from: /Library/Frameworks/Python.framework/Versions/3.6/Resources/Python.app/Contents/MacOS/Python
    Reason: image not found
PYTHONPATH was:
  /Users/harlanhutton/opt/anaconda3/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip:/Users/harlanhutton/opt/anaconda3/lib/python3.8/site-packages/pyspark/python/lib/py4j-0.10.9-src.zip:/Users/harlanhutton/opt/anaconda3/lib/python3.8/site-packages/pyspark/jars/spark-core_2.12-3.1.1.jar
org.apache.spark.SparkException: EOFException occurred while reading the port number 

In [60]:
labels.collect()

[(48884, 45092),
 (1757, 5219),
 (11827, 37),
 (50608, 12814),
 (50608, 63),
 (54627, 18967),
 (35076, 905),
 (54627, 2269),
 (54627, 6349),
 (35076, 650),
 (48884, 15218),
 (11827, 56086),
 (1757, 9113),
 (50608, 19998),
 (28588, 96233),
 (4576, 370),
 (1757, 4),
 (54627, 94),
 (50608, 35043),
 (48884, 106998),
 (54627, 9436),
 (54627, 7718),
 (28588, 412),
 (11827, 75104),
 (54627, 26177),
 (9831, 5537),
 (4576, 298),
 (48884, 37908),
 (11827, 435),
 (28588, 395),
 (28588, 4),
 (11827, 43290),
 (54627, 58462),
 (54627, 106998),
 (1757, 1926),
 (50608, 22201),
 (50608, 1155),
 (48884, 342),
 (16901, 248),
 (11827, 2739),
 (11827, 508),
 (28588, 28137),
 (11827, 101),
 (48884, 8311),
 (11827, 43393),
 (11827, 869),
 (28588, 10185),
 (54627, 49529),
 (48884, 5665),
 (28588, 12765),
 (48884, 14729),
 (16999, 106998),
 (11827, 1984),
 (11827, 106998),
 (54627, 39283),
 (28588, 66),
 (16999, 29),
 (11827, 5251),
 (28588, 106998),
 (11827, 1534),
 (35076, 4138),
 (11827, 236),
 (48884, 6),
